# Keypoint(Patch) Description

### Import packages

In [1]:
from __future__ import division, print_function
import glob
import os
import cv2
import PIL
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn.init
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.autograd import Variable
from copy import deepcopy, copy
from config_profile import args
from Utils import cv2_scale36, cv2_scale, np_reshape, np_reshape64
from Utils import L2Norm, cv2_scale, np_reshape
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

### Build the deep network: DesNet

In [2]:
# This is the CNN3 architecture
class DesNet(nn.Module):
    """DesdNet model definition
    """
    def __init__(self):
        super(DesNet, self).__init__()
        self.features = nn.Sequential(
            
            nn.Conv2d(1, 32, kernel_size=3, padding=1, bias = False),
            nn.BatchNorm2d(32, affine=False),
            nn.ReLU(),
            
            nn.Conv2d(32, 32, kernel_size=3, padding=1, bias = False),
            nn.BatchNorm2d(32, affine=False),
            nn.ReLU(),
            
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1, bias = False),
            nn.BatchNorm2d(64, affine=False),
            nn.ReLU(),
            
            nn.Conv2d(64, 64, kernel_size=3, padding=1, bias = False),
            nn.BatchNorm2d(64, affine=False),
            nn.ReLU(),
            
            nn.Conv2d(64, 128, kernel_size=3, stride=2,padding=1, bias = False),
            nn.BatchNorm2d(128, affine=False),
            nn.ReLU(),
            
            nn.Conv2d(128, 128, kernel_size=3, padding=1, bias = False),
            nn.BatchNorm2d(128, affine=False),
            nn.ReLU(),
            
            nn.Dropout(0.3),
            nn.Conv2d(128, 128, kernel_size=8, bias = False),
            nn.BatchNorm2d(128, affine=False),
        )
        self.features.apply(weights_init)
        return
    
    def input_norm(self,x):
        flat = x.view(x.size(0), -1)
        mp = torch.mean(flat, dim=1)
        sp = torch.std(flat, dim=1) + 1e-7
        return (x - mp.detach().unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand_as(x)) / sp.detach().unsqueeze(-1).unsqueeze(-1).unsqueeze(1).expand_as(x)
    
    def forward(self, input):
        x_features = self.features(self.input_norm(input))
        x = x_features.view(x_features.size(0), -1)
        return L2Norm()(x)

    
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.orthogonal_(m.weight.data, gain=0.6)
        try:
            nn.init.constant(m.bias.data, 0.01)
        except:
            pass
    return

In [ ]:
# load network
# from descriptor import DesNet
model = DesNet()
if args.cuda:
    model.cuda()

## Load network weights

In [ ]:
trained_weight_path = "checkpoint.pth" # suppose you are using checkpoint_4.pth
test_model = DesNet()
if args.cuda:
    test_model.cuda()
trained_weight = torch.load(trained_weight_path)['state_dict']
test_model.load_state_dict(trained_weight)
test_model.eval()

## Load raw patch files

In [ ]:
patches_dir = "train_patches.pth"
patches = torch.load(patches_dir)
print(patches.shape)
patches = patches.view(-1, 1, 32, 32).cuda()
print(patches.shape)


patches_dir = "test_patches.pth"
patches2 = torch.load(patches_dir)
print(patches2.shape)
patches2 = patches2.view(-1, 1, 32, 32).cuda()
print(patches2.shape)

<a id='get_deep_features_module_cell'></a>
### Get deep features


In [6]:
features = test_model(patches)
print(features.shape)
features = features.view(100, 5, 128).cpu().data  #x = NUM , y = keypoint, z = size
print(features.shape)

features2 = test_model(patches2)
print(features2.shape)
features2 = features2.view(50, 5, 128).cpu().data  #x = NUM , y = keypoint, z = size
print(features2.shape)

torch.Size([1700, 128])
torch.Size([100, 17, 128])
torch.Size([850, 128])
torch.Size([50, 17, 128])


## Save features

In [7]:
# save to file, with the name of features_CNN*.pth
features_dir = "train_features.pth"
torch.save(features, features_dir)

features_dir = "test_features.pth"
torch.save(features2, features_dir)

## Load and Test

In [8]:
features = torch.load("train_features.pth")
print(features[0][2])
fearutes2 = torch.load("test_features.pth")
print(features2[0][2])

tensor([ 2.8929e-02, -3.4354e-02,  1.7163e-02,  1.2051e-01, -9.7296e-02,
         5.8959e-02,  1.6133e-02, -8.1970e-02, -4.7363e-02, -9.5427e-03,
         1.3860e-01, -2.4629e-02,  5.6087e-02, -3.4598e-02,  9.3037e-02,
         1.7789e-01,  3.5172e-02, -1.2053e-01, -1.8533e-01,  5.4706e-02,
         6.5915e-02, -3.8367e-02,  8.7116e-04,  1.5862e-01, -5.3844e-02,
        -1.4471e-01,  1.0705e-02,  2.9205e-02, -9.9659e-02, -8.9973e-02,
        -1.8374e-01,  4.3521e-02,  9.1172e-02,  5.1403e-02,  8.2082e-03,
        -1.3303e-01,  7.8862e-02,  6.5711e-03, -9.8523e-02,  6.1728e-05,
        -5.0307e-02,  3.5834e-03,  4.2761e-02,  4.8837e-03,  7.2475e-02,
        -1.5026e-01,  3.3830e-02, -5.4504e-02, -1.0197e-01, -5.8610e-02,
        -2.1646e-01,  9.8381e-02,  1.4467e-02,  2.1669e-02,  9.2268e-02,
         1.6890e-02,  7.9713e-02, -3.3049e-02,  2.8121e-02, -4.8997e-02,
        -1.9121e-02, -6.0857e-03, -1.1361e-01, -7.6787e-02,  5.2484e-03,
         1.4322e-02,  9.3462e-02,  5.8809e-02, -5.5